## Import the relevant packages

In [1]:
import numpy as np
#import tensorflow as tf
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import input_data
#from tensorflow.examples.tutorials.mnist import input_data

# TensorFLow includes a data provider for MNIST that we'll use.
# This function automatically downloads the MNIST dataset to the chosen directory. 
# The dataset is already split into training, validation, and test subsets. 
# Furthermore, it preprocess it into a particularly simple and useful format.
# Every 28x28 image is flattened into a vector of length 28x28=784, where every value
# corresponds to the intensity of the color of the corresponding pixel.
# The samples are grayscale (but standardized from 0 to 1), so a value close to 0 is almost white and a value close to
# 1 is almost purely black. This representation (flattening the image row by row into
# a vector) is slightly naive but as you'll see it works surprisingly well.
# Since this is a classification problem, our targets are categorical.
# Recall from the lecture on that topic that one way to deal with that is to use one-hot encoding.
# With it, the target for each individual sample is a vector of length 10
# which has nine 0s and a single 1 at the position which corresponds to the correct answer.
# For instance, if the true answer is "1", the target will be [0,0,0,1,0,0,0,0,0,0] (counting from 0).
# Have in mind that the very first time you execute this command it might take a little while to run
# because it has to download the whole dataset. Following commands only extract it so they're faster.
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

#mnist = tf.keras.datasets.mnist.load_data()

#(train_images, train_labels) = tf.keras.datasets.mnist.load_data()

Instructions for updating:
non-resource variables are not supported in the long term
Instructions for updating:
Please use alternatives such as: tensorflow_datasets.load('mnist')
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/_DataSet.py from tensorflow/models.


In [20]:
mnist

_Datasets(train=<input_data._DataSet object at 0x127741e80>, validation=<input_data._DataSet object at 0x129ea0da0>, test=<input_data._DataSet object at 0x129ea0cf8>)

In [21]:
mnist['train']

TypeError: tuple indices must be integers or slices, not str

## Model

### Outline the model

In [4]:
input_size = 784
output_size = 10
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 100

# Reset any variables left in memory from previous runs.
tf.reset_default_graph()

# As in the previous example - declare placeholders where the data will be fed into.
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

# Weights and biases for the first linear combination between the inputs and the first hidden layer.
# Use get_variable in order to make use of the default TensorFlow initializer which is Xavier.
weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size])
biases_1 = tf.get_variable("biases_1", [hidden_layer_size])

# Operation between the inputs and the first hidden layer.
# We've chosen ReLu as our activation function. You can try playing with different non-linearities.
outputs_1 = tf.nn.relu(tf.matmul(inputs, weights_1) + biases_1)

# Weights and biases for the second linear combination.
# This is between the first and second hidden layers.
weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size])
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])

# Operation between the first and the second hidden layers. Again, we use ReLu.
outputs_2 = tf.nn.relu(tf.matmul(outputs_1, weights_2) + biases_2)

# Weights and biases for the final linear combination.
# That's between the second hidden layer and the output layer.
weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])

# Operation between the second hidden layer and the final output.
# Notice we have not used an activation function because we'll use the trick to include it directly in 
# the loss function. This works for softmax and sigmoid with cross entropy.
outputs = tf.matmul(outputs_2, weights_3) + biases_3

#

In [5]:
# Calculate the loss function for every output/target pair.
# The function used is the same as applying softmax to the last layer and then calculating cross entropy
# with the function we've seen in the lectures. This function, however, combines them in a clever way, 
# which makes it both faster and more numerically stable (when dealing with very small numbers).
# Logits here means: unscaled probabilities (so, the outputs, before they are scaled by the softmax)
# Naturally, the labels are the targets.
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)   # Applies Softmax Function

# Get the average loss
mean_loss = tf.reduce_mean(loss)

# Define the optimization step. Using adaptive optimizers such as Adam in TensorFlow
# is as simple as that.
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

# Get a 0 or 1 for every input in the batch indicating whether it output the correct answer out of the 10.
out_equals_target = tf.equal(tf.argmax(outputs, 1), tf.argmax(targets, 1))  # tf.argmax(outputs, 1) , 1 is for axis


# Get the average accuracy of the outputs.
accuracy = tf.reduce_mean(tf.cast(out_equals_target, tf.float32))

print("===========", out_equals_target, accuracy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

=========== Tensor("Equal:0", shape=(?,), dtype=bool) Tensor("Mean_1:0", shape=(), dtype=float32)


In [6]:
# Declare the session variable.
sess = tf.InteractiveSession()

# Initialize the variables. Default initializer is Xavier.
initializer = tf.global_variables_initializer()
sess.run(initializer)



In [7]:
# Batching
batch_size = 100

# Calculate the number of batches per epoch for the training set.
batches_number = mnist.train._num_examples // batch_size

# Basic early stopping. Set a miximum number of epochs.
max_epochs = 15

# Keep track of the validation loss of the previous epoch.
# If the validation loss becomes increasing, we want to trigger early stopping.
# We initially set it at some arbitrarily high number to make sure we don't trigger it
# at the first epoch
prev_validation_loss = 9999999.

In [8]:
# Create a loop for the epochs. Epoch_counter is a variable which automatically starts from 0.
for epoch_counter in range(max_epochs):
    
    # Keep track of the sum of batch losses in the epoch.
    curr_epoch_loss = 0.
    
    # Iterate over the batches in this epoch.
    for batch_counter in range(batches_number):
        
        # Input batch and target batch are assigned values from the train dataset, given a batch size
        input_batch, target_batch = mnist.train.next_batch(batch_size)
        
        # Run the optimization step and get the mean loss for this batch.
        # Feed it with the inputs and the targets we just got from the train dataset
        _, batch_loss = sess.run([optimize, mean_loss], 
            feed_dict={inputs: input_batch, targets: target_batch})
        
        # Increment the sum of batch losses.
        curr_epoch_loss += batch_loss
    
    # So far curr_epoch_loss contained the sum of all batches inside the epoch
    # We want to find the average batch losses over the whole epoch
    # The average batch loss is a good proxy for the current epoch loss
    curr_epoch_loss /= batches_number
    
    # At the end of each epoch, get the validation loss and accuracy
    # Get the input batch and the target batch from the validation dataset
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    
    # Run without the optimization step (simply forward propagate)
    validation_loss, validation_accuracy = sess.run([mean_loss, accuracy], 
        feed_dict={inputs: input_batch, targets: target_batch})
    
    # Print statistics for the current epoch
    # Epoch counter + 1, because epoch_counter automatically starts from 0, instead of 1
    # We format the losses with 3 digits after the dot
    # We format the accuracy in percentages for easier interpretation
    print('Epoch '+str(epoch_counter+1)+
          '. Mean loss: '+'{0:.3f}'.format(curr_epoch_loss)+
          '. Validation loss: '+'{0:.3f}'.format(validation_loss)+
          '. Validation accuracy: '+'{0:.2f}'.format(validation_accuracy * 100.)+'%')
    
    # Trigger early stopping if validation loss begins increasing.
    if validation_loss > prev_validation_loss:
        print("BREAKING")
        print(validation_loss, prev_validation_loss)
        break
        
    # Store this epoch's validation loss to be used as previous validation loss in the next iteration.
    prev_validation_loss = validation_loss

# Not essential, but it is nice to know when the algorithm stopped working in the output section, rather than check the kernel
print('End of training.')

Epoch 1. Mean loss: 0.332. Validation loss: 0.160. Validation accuracy: 95.38%
Epoch 2. Mean loss: 0.137. Validation loss: 0.131. Validation accuracy: 95.98%
Epoch 3. Mean loss: 0.097. Validation loss: 0.098. Validation accuracy: 97.14%
Epoch 4. Mean loss: 0.073. Validation loss: 0.087. Validation accuracy: 97.38%
Epoch 5. Mean loss: 0.056. Validation loss: 0.076. Validation accuracy: 97.66%
Epoch 6. Mean loss: 0.046. Validation loss: 0.082. Validation accuracy: 97.56%
BREAKING
0.08196368 0.07554064
End of training.


In [17]:
mnist.train

In [14]:
target_batch.shape

(10000, 10)

In [10]:
input_batch, target_batch = mnist.test.next_batch(mnist.test._num_examples)
    
# Run without the optimization step (simply forward propagate)
test_accuracy = sess.run([accuracy], feed_dict={inputs: input_batch, targets: target_batch})
test_accuracy_percent = test_accuracy[0] * 100
print('Test Accuracy: '+'{0:.2f}'.format(test_accuracy_percent) + '%')
    

Test Accuracy: 97.47%
